# Databricks Excel ファイル読み込み機能テスト

このノートブックでは、Databricks Runtime 17.1以上で利用可能な組み込みExcel読み込み機能をテストします。

**前提条件**
- Databricks Runtime 17.1 以上
- ワークスペース管理者がプレビュー機能を有効化済み

**参考ドキュメント**: https://docs.databricks.com/aws/ja/query/formats/excel

## 0. セットアップ: サンプルファイルのアップロード

事前に `databricks_excel_sample.xlsx` をVolume または DBFS にアップロードしてください。

In [0]:
# ファイルパスを設定（環境に合わせて変更してください）
# Unity Catalog Volume を使う場合
EXCEL_FILE_PATH = "/Volumes/takaakiyayoi_catalog/ai_functions/unstructured/input/databricks_excel_sample.xlsx"

# DBFS を使う場合（レガシー）
# EXCEL_FILE_PATH = "dbfs:/FileStore/databricks_excel_sample.xlsx"

## 1. 基本的な読み込み（最初のシート全体）

In [0]:
# デフォルト: 最初のシートの左上から右下の空でないセルまで読み込み
df_basic = spark.read.excel(EXCEL_FILE_PATH)

display(df_basic)

_c0,_c1,_c2,_c3,_c4,_c5,_c6
注文ID,商品名,カテゴリ,数量,単価,売上金額,注文日
ORD001,ノートPC,電子機器,2,89000,178000,2024-01-15
ORD002,マウス,周辺機器,10,2500,25000,2024-01-18
ORD003,キーボード,周辺機器,5,8000,40000,2024-02-03
ORD004,モニター,電子機器,3,45000,135000,2024-02-10
ORD005,USBケーブル,アクセサリ,20,500,10000,2024-02-22
ORD006,ヘッドセット,周辺機器,8,12000,96000,2024-03-05
ORD007,タブレット,電子機器,4,65000,260000,2024-03-12
ORD008,プリンター,電子機器,2,35000,70000,2024-03-20
null,null,合計:,54,null,814000,null


In [0]:
# スキーマを確認
df_basic.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)



## 2. ヘッダー行の指定（headerRows）

`headerRows` オプションで1行目をカラム名として認識させます。

In [0]:
# headerRows=1 でヘッダー行を列名として使用
df_with_header = (
    spark.read
    .option("headerRows", 1)
    .excel(EXCEL_FILE_PATH)
)

display(df_with_header)

注文ID,商品名,カテゴリ,数量,単価,売上金額,注文日
ORD001,ノートPC,電子機器,2,89000,178000,2024-01-15T00:00:00
ORD002,マウス,周辺機器,10,2500,25000,2024-01-18T00:00:00
ORD003,キーボード,周辺機器,5,8000,40000,2024-02-03T00:00:00
ORD004,モニター,電子機器,3,45000,135000,2024-02-10T00:00:00
ORD005,USBケーブル,アクセサリ,20,500,10000,2024-02-22T00:00:00
ORD006,ヘッドセット,周辺機器,8,12000,96000,2024-03-05T00:00:00
ORD007,タブレット,電子機器,4,65000,260000,2024-03-12T00:00:00
ORD008,プリンター,電子機器,2,35000,70000,2024-03-20T00:00:00
null,null,合計:,54,null,814000,null


In [0]:
# 列名がヘッダーから取得されていることを確認
print("列名:", df_with_header.columns)

列名: ['注文ID', '商品名', 'カテゴリ', '数量', '単価', '売上金額', '注文日']


## 3. 特定シートの読み込み（dataAddress）

`dataAddress` オプションでシート名やセル範囲を指定できます。

In [0]:
# 「顧客マスタ」シートを読み込み
df_customers = (
    spark.read
    .option("headerRows", 1)
    .option("dataAddress", "顧客マスタ")
    .excel(EXCEL_FILE_PATH)
)

display(df_customers)

顧客ID,顧客名,地域,登録日,会員ランク
C001,株式会社ABC,東京,2022-04-01T00:00:00,ゴールド
C002,DEF商事,大阪,2022-08-15T00:00:00,シルバー
C003,GHI工業,名古屋,2023-01-10T00:00:00,ブロンズ
C004,JKLサービス,福岡,2023-06-20T00:00:00,シルバー
C005,MNOテック,札幌,2024-02-05T00:00:00,ゴールド


## 4. セル範囲の指定（dataAddress）

特定の範囲だけを読み込むことができます。

In [0]:
# 「カテゴリ別集計」シートのA3:D7範囲を読み込み（タイトル行をスキップ）
df_summary = (
    spark.read
    .option("headerRows", 1)
    .option("dataAddress", "カテゴリ別集計!A3:D7")
    .excel(EXCEL_FILE_PATH)
)

display(df_summary)

カテゴリ,Q1売上,Q2売上,年間合計
電子機器,580000,720000,1300000
周辺機器,220000,185000,405000
アクセサリ,45000,62000,107000
合計,845000,967000,1812000


In [0]:
# 売上データシートの特定範囲（B2:F5）を読み込み
df_partial = (
    spark.read
    .option("headerRows", 0)
    .option("dataAddress", "売上データ!B2:F5")
    .excel(EXCEL_FILE_PATH)
)

display(df_partial)

_c0,_c1,_c2,_c3,_c4
ノートPC,電子機器,2,89000,178000
マウス,周辺機器,10,2500,25000
キーボード,周辺機器,5,8000,40000
モニター,電子機器,3,45000,135000


## 5. シート一覧の取得（listSheets）

`operation` オプションで Excel ファイル内のシート一覧を取得できます。

In [0]:
# シート一覧を取得
df_sheets = (
    spark.read
    .option("operation", "listSheets")
    .excel(EXCEL_FILE_PATH)
)

display(df_sheets)

sheetIndex,sheetName
0,売上データ
1,顧客マスタ
2,カテゴリ別集計


## 6. SQL での読み込み（read_files関数）

### 6.1 基本的なSQL読み込み

In [0]:
# SQLで読み込み（変数を使うためにPythonから実行）
df_sql = spark.sql(f"""
SELECT * FROM read_files(
    '{EXCEL_FILE_PATH}',
    format => 'excel',
    headerRows => 1,
    schemaEvolutionMode => 'none'
)
""")

display(df_sql)

注文ID,商品名,カテゴリ,数量,単価,売上金額,注文日
ORD001,ノートPC,電子機器,2,89000,178000,2024-01-15T00:00:00
ORD002,マウス,周辺機器,10,2500,25000,2024-01-18T00:00:00
ORD003,キーボード,周辺機器,5,8000,40000,2024-02-03T00:00:00
ORD004,モニター,電子機器,3,45000,135000,2024-02-10T00:00:00
ORD005,USBケーブル,アクセサリ,20,500,10000,2024-02-22T00:00:00
ORD006,ヘッドセット,周辺機器,8,12000,96000,2024-03-05T00:00:00
ORD007,タブレット,電子機器,4,65000,260000,2024-03-12T00:00:00
ORD008,プリンター,電子機器,2,35000,70000,2024-03-20T00:00:00
null,null,合計:,54,null,814000,null


### 6.2 特定シート・範囲をSQLで読み込み

In [0]:
df_sql_range = spark.sql(f"""
SELECT * FROM read_files(
    '{EXCEL_FILE_PATH}',
    format => 'excel',
    headerRows => 1,
    dataAddress => '顧客マスタ!A1:E6',
    schemaEvolutionMode => 'none'
)
""")

display(df_sql_range)

顧客ID,顧客名,地域,登録日,会員ランク
C001,株式会社ABC,東京,2022-04-01T00:00:00,ゴールド
C002,DEF商事,大阪,2022-08-15T00:00:00,シルバー
C003,GHI工業,名古屋,2023-01-10T00:00:00,ブロンズ
C004,JKLサービス,福岡,2023-06-20T00:00:00,シルバー
C005,MNOテック,札幌,2024-02-05T00:00:00,ゴールド


### 6.3 SQLでシート一覧を取得

In [0]:
df_sql_sheets = spark.sql(f"""
SELECT * FROM read_files(
    '{EXCEL_FILE_PATH}',
    operation => 'listSheets',
    schemaEvolutionMode => 'none'
)
""")

display(df_sql_sheets)

sheetIndex,sheetName
0,売上データ
1,顧客マスタ
2,カテゴリ別集計


## 7. テーブルへの保存

In [0]:
# 一時ビューとして登録
df_with_header.createOrReplaceTempView("sales_temp")

In [0]:
%sql
-- 一時ビューの内容を確認
SELECT * FROM sales_temp

注文ID,商品名,カテゴリ,数量,単価,売上金額,注文日
ORD001,ノートPC,電子機器,2,89000,178000,2024-01-15T00:00:00
ORD002,マウス,周辺機器,10,2500,25000,2024-01-18T00:00:00
ORD003,キーボード,周辺機器,5,8000,40000,2024-02-03T00:00:00
ORD004,モニター,電子機器,3,45000,135000,2024-02-10T00:00:00
ORD005,USBケーブル,アクセサリ,20,500,10000,2024-02-22T00:00:00
ORD006,ヘッドセット,周辺機器,8,12000,96000,2024-03-05T00:00:00
ORD007,タブレット,電子機器,4,65000,260000,2024-03-12T00:00:00
ORD008,プリンター,電子機器,2,35000,70000,2024-03-20T00:00:00
null,null,合計:,54,null,814000,null


In [0]:
# Delta テーブルとして保存（カタログ/スキーマは環境に合わせて変更）
# df_with_header.write.mode("overwrite").saveAsTable("your_catalog.your_schema.excel_sales_data")

## 8. COPY INTO でのインポート

In [0]:
%sql
-- テーブルが存在しない場合は作成（環境に合わせて変更）
-- CREATE TABLE IF NOT EXISTS your_catalog.your_schema.excel_demo_table;

-- COPY INTO your_catalog.your_schema.excel_demo_table
-- FROM '/Volumes/your_catalog/your_schema/your_volume/databricks_excel_sample.xlsx'
-- FILEFORMAT = EXCEL
-- FORMAT_OPTIONS ('headerRows' = '1', 'mergeSchema' = 'true')
-- COPY_OPTIONS ('mergeSchema' = 'true');

## 9. Auto Loader でのストリーミング読み込み

継続的にExcelファイルを監視・読み込みする場合に使用します。

In [0]:
# ストリーミング読み込み（実行する場合はコメントを外してください）
# checkpoint_path = "/Volumes/your_catalog/your_schema/your_volume/checkpoints/excel_stream"
# schema_path = "/Volumes/your_catalog/your_schema/your_volume/schemas/excel_stream"
# source_path = "/Volumes/your_catalog/your_schema/your_volume/excel_files/"

# df_stream = (
#     spark.readStream
#     .format("cloudFiles")
#     .option("cloudFiles.format", "excel")
#     .option("cloudFiles.inferColumnTypes", True)
#     .option("headerRows", 1)
#     .option("cloudFiles.schemaLocation", schema_path)
#     .option("cloudFiles.schemaEvolutionMode", "none")
#     .load(source_path)
# )

# # Delta テーブルに書き込み
# (
#     df_stream.writeStream
#     .format("delta")
#     .option("mergeSchema", "true")
#     .option("checkpointLocation", checkpoint_path)
#     .table("your_catalog.your_schema.excel_streaming_table")
# )

## 10. 数式の評価確認

Excelの数式は計算済みの値として取り込まれます。

In [0]:
# 売上金額列（数式 =D*E で計算）が正しく取り込まれているか確認
df_formula_check = (
    spark.read
    .option("headerRows", 1)
    .option("dataAddress", "売上データ!A1:F9")
    .excel(EXCEL_FILE_PATH)
)

display(df_formula_check)

注文ID,商品名,カテゴリ,数量,単価,売上金額
ORD001,ノートPC,電子機器,2,89000,178000
ORD002,マウス,周辺機器,10,2500,25000
ORD003,キーボード,周辺機器,5,8000,40000
ORD004,モニター,電子機器,3,45000,135000
ORD005,USBケーブル,アクセサリ,20,500,10000
ORD006,ヘッドセット,周辺機器,8,12000,96000
ORD007,タブレット,電子機器,4,65000,260000
ORD008,プリンター,電子機器,2,35000,70000


In [0]:
# 数量 × 単価 = 売上金額 の検証
from pyspark.sql.functions import col

df_verify = df_formula_check.withColumn(
    "計算検証",
    col("数量") * col("単価") == col("売上金額")
)

display(df_verify.select("商品名", "数量", "単価", "売上金額", "計算検証"))

商品名,数量,単価,売上金額,計算検証
ノートPC,2,89000,178000,true
マウス,10,2500,25000,true
キーボード,5,8000,40000,true
モニター,3,45000,135000,true
USBケーブル,20,500,10000,true
ヘッドセット,8,12000,96000,true
タブレット,4,65000,260000,true
プリンター,2,35000,70000,true


## 11. 複数シートの一括処理

`listSheets` で取得したシート名を使って、全シートを処理できます。

In [0]:
# シート一覧を取得
sheets_df = (
    spark.read
    .option("operation", "listSheets")
    .excel(EXCEL_FILE_PATH)
)

sheet_names = [row.sheetName for row in sheets_df.collect()]
print("シート一覧:", sheet_names)

シート一覧: ['売上データ', '顧客マスタ', 'カテゴリ別集計']


In [0]:
# 各シートを読み込んで表示
for sheet_name in sheet_names:
    print(f"\n=== {sheet_name} ===")
    df = (
        spark.read
        .option("headerRows", 1)
        .option("dataAddress", sheet_name)
        .excel(EXCEL_FILE_PATH)
    )
    df.show()


=== 売上データ ===
+------+------------+----------+----+-----+--------+-------------------+
|注文ID|      商品名|  カテゴリ|数量| 単価|売上金額|             注文日|
+------+------------+----------+----+-----+--------+-------------------+
|ORD001|    ノートPC|  電子機器|   2|89000|  178000|2024-01-15 00:00:00|
|ORD002|      マウス|  周辺機器|  10| 2500|   25000|2024-01-18 00:00:00|
|ORD003|  キーボード|  周辺機器|   5| 8000|   40000|2024-02-03 00:00:00|
|ORD004|    モニター|  電子機器|   3|45000|  135000|2024-02-10 00:00:00|
|ORD005| USBケーブル|アクセサリ|  20|  500|   10000|2024-02-22 00:00:00|
|ORD006|ヘッドセット|  周辺機器|   8|12000|   96000|2024-03-05 00:00:00|
|ORD007|  タブレット|  電子機器|   4|65000|  260000|2024-03-12 00:00:00|
|ORD008|  プリンター|  電子機器|   2|35000|   70000|2024-03-20 00:00:00|
|  NULL|        NULL|     合計:|  54| NULL|  814000|               NULL|
+------+------------+----------+----+-----+--------+-------------------+


=== 顧客マスタ ===
+------+-----------+------+-------------------+----------+
|顧客ID|     顧客名|  地域|             登録日|会員ランク|
+------

## まとめ

| 機能 | オプション | 例 |
|------|-----------|-----|
| ヘッダー行指定 | `headerRows` | `1` (1行目をヘッダーとして使用) |
| シート/範囲指定 | `dataAddress` | `"Sheet1!A1:E10"` |
| シート一覧取得 | `operation` | `"listSheets"` |
| ストリーミング | `cloudFiles.format` | `"excel"` |

**制限事項**
- パスワード保護ファイルは非対応
- ヘッダー行は1行のみ対応
- 結合セルは左上のみ値が入る
- スキーマ進化は非対応（`schemaEvolutionMode="none"` が必要）